In [1]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle
import jdatetime
import matplotlib.pyplot as plt
import seaborn as sns
from bidi.algorithm import get_display
from arabic_reshaper import reshape
import arabic_reshaper
import geopandas
#starting date = 29 tir

In [2]:
df = pd.read_csv("job.csv")

C:\Users\Arian_Gh\AppData\Local\Temp\ipykernel_19400\1541966218.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("job.csv")


In [3]:
df.head()

,company_size,education,gender,salary,workdays,language,business_trip,industry,bonus,location,ad_name,softwares,military_services,experience,date,min_age,max_age
0,30,['کارشناسی| مالی و حسابداری'],ترجیحاً خانم,NaN,شنبه تا چهارشنبه 8 الی 16,['انگلیسی| متوسط ۵۰٪'],-,نفت، گاز و پتروشیمی,-,تهران، فاطمی,کارشناس حسابداری,"['Microsoft Word| متوسط', 'Microsoft Excel| مت...",NaN,NaN,1402-03-23,23.0,33.0
1,125,NaN,فقط آقا,12 - 10 میلیون تومان,شنبه تا چهارشنبه 8 تا 17:33، پنجشنبه 8:30 تا ...,NaN,-,تولیدی / صنعتی,پاداش -بیمه درمان تکمیلی,رباط کریم,کارگر ساده - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-26,23.0,45.0
2,5000+,NaN,تفاوتی ندارد,NaN,شنبه تا چهارشنبه 7 الی 16,NaN,-,بانکداری,وام -پاداش -بیمه درمان تکمیلی -تسهیلات تفری...,تهران، آرژانتین,کارشناس موشن گرافیست,"['Adobe Photoshop| پیشرفته', 'Adobe Illustrato...",اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-11,NaN,35.0
3,30,NaN,تفاوتی ندارد,20 - 15 میلیون تومان,10 تا 20,['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,پورسانت,تهران، جردن,مدیر اجرایی,"['Microsoft Word| متوسط', 'Microsoft Powerpoin...",NaN,NaN,1402-02-12,25.0,43.0
4,750,NaN,فقط آقا,NaN,شیفت شب ( 3 روز در هفته ),['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,بیمه درمان تکمیلی -اتاق بازی -ناهار -میان و...,کرج، حیدرآباد,کارشناس مرکز تماس ( شیفت شب) - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-04-06,20.0,30.0


In [4]:
def convert_salary(salary_str):
    if pd.isna(salary_str):
        return np.nan
    
    # Handle both versions of the "million toman" string in Persian
    s_cleaned = salary_str.replace(' میلیون تومان', '').replace(' ميليون تومان', '').replace(' Million Tomans', '')
    
    # Handle "Above 60" and "60 به بالا" cases
    if "Above" in s_cleaned:
        s_cleaned = s_cleaned.replace('Above ', '') + '-70'
    elif "به بالا" in s_cleaned:
        s_cleaned = s_cleaned.replace(' به بالا', '') + '-70'
    
    try:
        salaries = [float(s.strip()) for s in s_cleaned.split("-")]
        return sum(salaries)/len(salaries)
    except ValueError:
        # Print the problematic string for debugging
        print(f"Problematic string: {s_cleaned}")
        return np.nan

df['avg_salary'] = df['salary'].apply(convert_salary)

In [5]:
del df["salary"]

In [6]:
df.head()

,company_size,education,gender,workdays,language,business_trip,industry,bonus,location,ad_name,softwares,military_services,experience,date,min_age,max_age,avg_salary
0,30,['کارشناسی| مالی و حسابداری'],ترجیحاً خانم,شنبه تا چهارشنبه 8 الی 16,['انگلیسی| متوسط ۵۰٪'],-,نفت، گاز و پتروشیمی,-,تهران، فاطمی,کارشناس حسابداری,"['Microsoft Word| متوسط', 'Microsoft Excel| مت...",NaN,NaN,1402-03-23,23.0,33.0,NaN
1,125,NaN,فقط آقا,شنبه تا چهارشنبه 8 تا 17:33، پنجشنبه 8:30 تا ...,NaN,-,تولیدی / صنعتی,پاداش -بیمه درمان تکمیلی,رباط کریم,کارگر ساده - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-26,23.0,45.0,11.0
2,5000+,NaN,تفاوتی ندارد,شنبه تا چهارشنبه 7 الی 16,NaN,-,بانکداری,وام -پاداش -بیمه درمان تکمیلی -تسهیلات تفری...,تهران، آرژانتین,کارشناس موشن گرافیست,"['Adobe Photoshop| پیشرفته', 'Adobe Illustrato...",اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-11,NaN,35.0,NaN
3,30,NaN,تفاوتی ندارد,10 تا 20,['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,پورسانت,تهران، جردن,مدیر اجرایی,"['Microsoft Word| متوسط', 'Microsoft Powerpoin...",NaN,NaN,1402-02-12,25.0,43.0,17.5
4,750,NaN,فقط آقا,شیفت شب ( 3 روز در هفته ),['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,بیمه درمان تکمیلی -اتاق بازی -ناهار -میان و...,کرج، حیدرآباد,کارشناس مرکز تماس ( شیفت شب) - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-04-06,20.0,30.0,NaN


In [7]:
df_location = df.copy()

In [8]:
df_location["location"] = df_location["location"].apply(lambda x: str(x).split("،")[0] if x != "nan" else x)

In [9]:
df_location["location"] = df_location["location"].apply(lambda x: str(x).replace("امکان دورکاری",""))
df_location["location"] = df_location["location"].apply(lambda x: str(x).replace("امکان استخدام معلولین",""))
df_location["location"] = df_location["location"].apply(lambda x: str(x).replace("امکان جذب کارآموز",""))

In [10]:
df_location["location"].value_counts()

location
تهران                                    19400
اصفهان                                    1198
کرج                                       1067
مشهد                                       966
nan                                        603
                                         ...  
Alborz Industrial City                       1
سعادت آباد                                   1
Tehran/ Abas Abad(Beheshti)Internship        1
Tehran/ Gheytariyeh                          1
امیرکلا                                      1
Name: count, Length: 756, dtype: int64

In [12]:
import re
pattern = r'[a-zA-Z]'

# Apply the regex pattern to the 'ad_name' column and create a new column 'is_english'
df_location['is_english'] = df_location['gender'].apply(lambda x: bool(re.search(pattern, str(x))))

# Filter the DataFrame to only include English job ads
df_location = df_location[df_location['is_english']==False]

In [59]:
df2 = df_location.copy()
df2 = df2.reset_index(drop=True)
df2.head()

,company_size,education,gender,workdays,language,business_trip,industry,bonus,location,ad_name,softwares,military_services,experience,date,min_age,max_age,avg_salary,is_english
0,30,['کارشناسی| مالی و حسابداری'],ترجیحاً خانم,شنبه تا چهارشنبه 8 الی 16,['انگلیسی| متوسط ۵۰٪'],-,نفت، گاز و پتروشیمی,-,تهران، فاطمی,کارشناس حسابداری,"['Microsoft Word| متوسط', 'Microsoft Excel| مت...",NaN,NaN,1402-03-23,23.0,33.0,NaN,False
1,125,NaN,فقط آقا,شنبه تا چهارشنبه 8 تا 17:33، پنجشنبه 8:30 تا ...,NaN,-,تولیدی / صنعتی,پاداش -بیمه درمان تکمیلی,رباط کریم,کارگر ساده - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-26,23.0,45.0,11.0,False
2,5000+,NaN,تفاوتی ندارد,شنبه تا چهارشنبه 7 الی 16,NaN,-,بانکداری,وام -پاداش -بیمه درمان تکمیلی -تسهیلات تفری...,تهران، آرژانتین,کارشناس موشن گرافیست,"['Adobe Photoshop| پیشرفته', 'Adobe Illustrato...",اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-11,NaN,35.0,NaN,False
3,30,NaN,تفاوتی ندارد,10 تا 20,['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,پورسانت,تهران، جردن,مدیر اجرایی,"['Microsoft Word| متوسط', 'Microsoft Powerpoin...",NaN,NaN,1402-02-12,25.0,43.0,17.5,False
4,750,NaN,فقط آقا,شیفت شب ( 3 روز در هفته ),['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,بیمه درمان تکمیلی -اتاق بازی -ناهار -میان و...,کرج، حیدرآباد,کارشناس مرکز تماس ( شیفت شب) - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-04-06,20.0,30.0,NaN,False


In [61]:
df2["location"]

,company_size,education,gender,workdays,language,business_trip,industry,bonus,location,ad_name,softwares,military_services,experience,date,min_age,max_age,avg_salary,is_english
0,30,['کارشناسی| مالی و حسابداری'],ترجیحاً خانم,شنبه تا چهارشنبه 8 الی 16,['انگلیسی| متوسط ۵۰٪'],-,نفت، گاز و پتروشیمی,-,تهران، فاطمی,کارشناس حسابداری,"['Microsoft Word| متوسط', 'Microsoft Excel| مت...",NaN,NaN,1402-03-23,23.0,33.0,NaN,False
1,125,NaN,فقط آقا,شنبه تا چهارشنبه 8 تا 17:33، پنجشنبه 8:30 تا ...,NaN,-,تولیدی / صنعتی,پاداش -بیمه درمان تکمیلی,رباط کریم,کارگر ساده - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-26,23.0,45.0,11.0,False
2,5000+,NaN,تفاوتی ندارد,شنبه تا چهارشنبه 7 الی 16,NaN,-,بانکداری,وام -پاداش -بیمه درمان تکمیلی -تسهیلات تفری...,تهران، آرژانتین,کارشناس موشن گرافیست,"['Adobe Photoshop| پیشرفته', 'Adobe Illustrato...",اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-11,NaN,35.0,NaN,False
3,30,NaN,تفاوتی ندارد,10 تا 20,['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,پورسانت,تهران، جردن,مدیر اجرایی,"['Microsoft Word| متوسط', 'Microsoft Powerpoin...",NaN,NaN,1402-02-12,25.0,43.0,17.5,False
4,750,NaN,فقط آقا,شیفت شب ( 3 روز در هفته ),['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,بیمه درمان تکمیلی -اتاق بازی -ناهار -میان و...,کرج، حیدرآباد,کارشناس مرکز تماس ( شیفت شب) - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-04-06,20.0,30.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32814,125,['کارشناسی ارشد| شیمی'],تفاوتی ندارد,شنبه تا چهارشنبه 7:30 الی 16 و پنجشنبه ها یک د...,['انگلیسی| متوسط ۵۰٪'],-,تولیدی / صنعتی,-,ساوه,مسئول فنی (معاونت غذا و دارو),NaN,NaN,NaN,1402-03-21,22.0,50.0,NaN,False
32815,350,"['کاردانی| مهندسی برق', 'کاردانی| مهندسی مکانی...",فقط آقا,همه روزه 7:30 الی 16:00 - پنج شنبه ها یک هفته ...,['انگلیسی| متوسط ۵۰٪'],-,تولیدی / صنعتی,-,نظر آباد,تکنسین نگهداری و تعمیرات - آقا,NaN,NaN,NaN,1402-04-19,25.0,40.0,NaN,False
32816,750,['کارشناسی| مالی و حسابداری'],تفاوتی ندارد,شنبه تا چهارشنبه 8:00 الی 17:00,['انگلیسی| مقدماتی ۱۰٪'],-,دارو,وام -پاداش -پورسانت -بیمه درمان تکمیلی -اق...,تهران، تولید دارو,کارشناس حسابداری فروش,"['Microsoft Word| متوسط', 'Microsoft Powerpoin...",اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-07,24.0,37.0,NaN,False
32817,30,NaN,فقط خانم,9 - 17,NaN,-,تولیدی / صنعتی,وام -پاداش -ناهار,تهران، قلهک,کارمند اداری - خانم,"['Microsoft Excel| متوسط', 'Microsoft Word| مت...",NaN,NaN,1402-03-28,35.0,45.0,9.0,False
